In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# default_exp xgb_optuna_objective

# XGB Optuna Objective

> An API to create Optuna Ojbective

In [3]:
#hide
from nbdev.showdoc import *
from nbdev import *

In [4]:
# export
from tabular_ml_toolkit.dataframeloader import *
from tabular_ml_toolkit.preprocessor import *
from tabular_ml_toolkit.logger import *

In [5]:
# export
# hide
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss, f1_score, precision_score, recall_score
# for Optuna
import optuna
#for XGB
from xgboost import XGBRegressor, XGBClassifier


# for displaying diagram of pipelines 
from sklearn import set_config
set_config(display="diagram")

# for finding n_jobs in all sklearn estimators
from sklearn.utils import all_estimators
import inspect

# Just to compare fit times
import time

# for os specific settings
import os
import gc

In [6]:
# export

class XGB_Optuna_Objective:
    """
    Represent XGB Optuna Objective class
    
    Attributes:\n
    trial: optuna trial object \n
    use_gpu: list of prediction metrics
    """

    def __init__(self, X_train_np, y_train_np, X_valid_np, y_valid_np, xgb_model_type,
                 val_preds_metrics, xgb_eval_metric, use_gpu, verbose):
        
        #attributes
        self.X_train_np = X_train_np
        self.y_train_np = y_train_np
        self.X_valid_np = X_valid_np
        self.y_valid_np = y_valid_np
        self.val_preds_metrics = val_preds_metrics
        self.xgb_eval_metric = xgb_eval_metric
        self.xgb_model_type = xgb_model_type
        self.use_gpu = use_gpu
        self.verbose = verbose
        self.trial = None

    def __call__(self, trial):
        self.trial = trial
        
        #get_params here
        xgb_params = self.get_xgb_params(self.trial, use_gpu=self.use_gpu)
        early_stopping_rounds = xgb_params["early_stopping_rounds"]
        del xgb_params["early_stopping_rounds"]
        #xgb_params.update({'verbose':self.verbose})
        
        logger.info(f"final params {xgb_params}")
        
        # create xgb ml model
        model = self.xgb_model_type(
            random_state=42,
            eval_set=[(self.X_valid_np, self.y_valid_np)],
            eval_metric=self.xgb_eval_metric,
            use_label_encoder=False,
            #early_stopping_rounds=early_stopping_rounds,
            **xgb_params,
        )
    
        # Now fit
        logger.info("Training Started!")
        model.fit(self.X_train_np, self.y_train_np, verbose=True)
        logger.info("Training Ended!")
        
        gc.collect()
        # choose predict type based upon metrics type
        #TO-DO instead of single metrics use list of metrics and calculate mean using dict
        metric_result = {}
        metric = self.val_preds_metrics
        if ("log_loss" in str(metric.__name__)) or ("roc_auc_score" in str(metric.__name__)):
            #logger.info("Predicting Probablities!")
            preds_probs = model.predict_proba(self.X_valid_np)[:, 1]
            metric_result[str(metric.__name__)] = metric(self.y_valid_np, self.preds_probs)

        else:
            #logger.info("Predicting Score!")
            preds = model.predict(self.X_valid_np)
            metric_result[str(metric.__name__)] = metric(self.y_valid_np, preds)
        
        #now show value of all the given metrics
        for metric_name, metric_value in metric_result.items():
            logger.info(f"{metric_name}: {metric_value}")
        
        # choose return result for Optuna Optimization based upon metric type
        if self.xgb_eval_metric == "logloss":
            return metric_result["log_loss"]
        elif self.xgb_eval_metric == "mlogloss":
            return metric_result["accuracy_score"]
        elif self.xgb_eval_metric == "rmse":
            return metric_result["mean_absolute_error"]
        elif self.xgb_eval_metric == "auc":
            return metric_result["roc_auc_score"]
        else:
            return None        
        
    
    def __str__(self):
        """Returns human readable string reprsentation"""
        attr_str = ("X_train_np, X_valid_np, y_train_np, y_valid_np, use_gpu, verbose")
        return ("Optuna Objective object with attributes:"+attr_str)
    
    def __repr__(self):
        return self.__str__()

    #helper methods
    #Method to get xgb_params
    def get_xgb_params(self, trial, use_gpu):
        params = {
            "learning_rate": trial.suggest_float("learning_rate", 1e-2, 0.25, log=True),
            "n_estimators": trial.suggest_categorical("n_estimators", [100, 500, 1000]),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 100.0, log=True),
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 100.0, log=True),
            "subsample": trial.suggest_float("subsample", 0.1, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
            "max_depth": trial.suggest_int("max_depth", 1, 9),
            "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 100, 500),
        }
        if use_gpu:
            params["tree_method"] = "gpu_hist"
            params["gpu_id"] = 0
            params["predictor"] = "gpu_predictor"
        else:
            params["tree_method"] = trial.suggest_categorical("tree_method", ["exact", "approx", "hist"])
            params["booster"] = trial.suggest_categorical("booster", ["gbtree", "gblinear"])
            if params["booster"] == "gbtree":
                params["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
                params["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

        return params
                
    # core methods

In [7]:
# hide
# run the script to build 

from nbdev.export import notebook2script; notebook2script()

Converted 00_dataframeloader.ipynb.
Converted 01_preprocessor.ipynb.
Converted 02_tmlt.ipynb.
Converted 04_xgb_optuna_objective.ipynb.
Converted Kaggle_TPS_Dec_ENSEMBLE_Clean.ipynb.
Converted Kaggle_TPS_Dec_Tutorial-Meta.ipynb.
Converted Kaggle_TPS_Dec_Tutorial_XGB.ipynb.
Converted Kaggle_TPS_Nov_Tutorial.ipynb.
Converted index.ipynb.
Converted logger.ipynb.
Converted utility.ipynb.
Converted xgb_tabular_ml_toolkit.ipynb.
